## PhysionetMI

Load data of all 109 subjects.

Only the runs of "hands vs feet" motor imagery are loaded [runs 6, 10, 14]. Data is extracted as mne.io.Raw objects, each object representing the concatenated runs of a single subject.

In [ ]:
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage
from sklearn.pipeline import Pipeline
from scripts.raw_preprocessing import FilterRaw, RemoveArtifacts, SelectChannels, Epochify

runs = [6, 10, 14]  # motor imagery: hands vs feet
montage = make_standard_montage("standard_1005")

channels = ["C3", "C4", "P3", "P4", "T7", "T8", "P7", "P8"] # These positions of the 10-10 system are equivalent to our positions in the 10-20 system
event_ids = ['rest', 'hands', 'feet']

physionet_preprocessing_pipeline = Pipeline([
    ("filter", FilterRaw()),
    ("remove_artifacts", RemoveArtifacts()),
    # ("select_channels", SelectChannels(channels=channels)),
    ("epochify", Epochify(event_ids=event_ids, channels=channels)),
])

physionet_subjects = {}

for subject in range(1, 110):
    f_names = eegbci.load_data(subject, runs=runs, path="datasets", update_path=True)
    raw = concatenate_raws([read_raw_edf(f_name, preload=True) for f_name in f_names])

    if raw.info["sfreq"] == 160:
        eegbci.standardize(raw)  # set channel names
        raw.set_montage(montage)
        raw.annotations.rename(dict(T0="rest", T1="hands", T2="feet"))
        raw.set_eeg_reference(projection=True)

        subject_epochs = physionet_preprocessing_pipeline.fit_transform(raw)

        physionet_subjects[f"P{subject:03}"] = subject_epochs

# Save the preprocessed data in /data/physionet.pkl
import pickle
with open("pickled_data/physionet.pkl", "wb") as f:
    pickle.dump(physionet_subjects, f)

## Schirrmeister 2017

In [4]:
from moabb.datasets import Schirrmeister2017
from sklearn.pipeline import Pipeline
from scripts.raw_preprocessing import FilterRaw, RemoveArtifacts, Epochify, Resampler
from scripts.utils import load_single_moabb_subject

dataset = Schirrmeister2017()

event_ids = ['feet', 'left_hand', 'rest', 'right_hand']
channels = ["C3", "C4", "P3", "P4", "T7", "T8", "P7", "P8"] # These positions of the 10-10 system are equivalent to our positions in the 10-20 system

schirrmeister_preprocessing_pipeline = Pipeline([
    ("filter", FilterRaw()),
    ("remove_artifacts", RemoveArtifacts(n_components=8)),
    ("epochify", Epochify(event_ids=event_ids, channels=channels)),
    ("resample", Resampler(sfreq=160)),
])

schirrmeister_subjects = {}

for subject in dataset.subject_list:
    raw = load_single_moabb_subject(dataset, subject)

    raw.drop_channels([ch for ch in raw.ch_names if ch not in channels])

    subject_epochs = schirrmeister_preprocessing_pipeline.fit_transform(raw)

    schirrmeister_subjects[f"S{subject:03}"] = subject_epochs

# Save the preprocessed data in /data/schirrmeister.pkl
import pickle

with open("pickled_data/schirrmeister.pkl", "wb") as f:
    pickle.dump(schirrmeister_subjects, f)


Extracting EDF parameters from F:\zeyad\Documents\repos\general-bci-tl-classifier\datasets\MNE-schirrmeister2017-data\robintibor\high-gamma-dataset\raw\master\data\train\1.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as type EEG (renamed to 'Fpz').
Channel 'EEG F7' recognized as type EEG (renamed to 'F7').
Channel 'EEG F3' recognized as type EEG (renamed to 'F3').
Channel 'EEG Fz' recognized as type EEG (renamed to 'Fz').
Channel 'EEG F4' recognized as type EEG (renamed to 'F4').
Channel 'EEG F8' recognized as type EEG (renamed to 'F8').
Channel 'EEG FC5' recognized as type EEG (renamed to 'FC5').
Channel 'EEG FC1' recognized as type EEG (renamed to 'FC1').
Channel 'EEG FC2' recognized as type EEG (renamed to 'FC2').
Channel 'EEG FC6' recognized as type EEG (renamed to 'FC6').
Channel 'EEG M1' recognized as type EEG (renamed to 'M1').
Channel 'EEG T7' recog

f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:69: RuntimeWarning: Using n_components=8 (resulting in n_components_=8) may lead to an unstable mixing matrix estimation because the ratio between the largest (3.2) and smallest (2.6e-29) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 7
  ica.fit(raw)
f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:72: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ica_labels = label_components(raw, ica, method='iclabel')


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'left_hand', 'rest', 'right_hand']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 480 events and 2501 original time points ...
0 bad epochs dropped
Extracting EDF parameters from F:\zeyad\Documents\repos\general-bci-tl-classifier\datasets\MNE-schirrmeister2017-data\robintibor\high-gamma-dataset\raw\master\data\train\2.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as 

f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:69: RuntimeWarning: Using n_components=8 (resulting in n_components_=8) may lead to an unstable mixing matrix estimation because the ratio between the largest (3.4) and smallest (1.8e-28) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 7
  ica.fit(raw)


Fitting ICA took 228.1s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'left_hand', 'rest', 'right_hand']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
960 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 960 events and 2501 original time points ...
0 bad epochs dropped
Extracting EDF parameters from F:\zeyad\Documents\repos\general-bci-tl-classifier\datasets\MNE-schirrmeister2017-data\robintibor\high-gamma-dataset\raw\master\data\train\14.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as type EEG (renamed to 'Fpz').
Channel 'EEG F7' recognized as type EEG (renamed t

f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:69: RuntimeWarning: Using n_components=8 (resulting in n_components_=8) may lead to an unstable mixing matrix estimation because the ratio between the largest (2.6) and smallest (2.7e-29) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 7
  ica.fit(raw)


Fitting ICA took 309.4s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:72: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ica_labels = label_components(raw, ica, method='iclabel')


Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'left_hand', 'rest', 'right_hand']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
1040 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1040 events and 2501 original time points ...
0 bad epochs dropped


## Weibo 2014

In [11]:
from moabb.datasets import Weibo2014
from scripts.raw_preprocessing import FilterRaw, RemoveArtifacts, Epochify, Resampler
from scripts.utils import load_single_moabb_subject

dataset = Weibo2014()

event_ids = ['feet', 'hands', 'left_hand', 'left_hand_right_foot', 'rest', 'right_hand', 'right_hand_left_foot']
channels = ["C3", "C4", "P3", "P4", "T7", "T8", "P7", "P8"] # These positions of the 10-10 system are equivalent to our positions in the 10-20 system
weibo_preprocessing_pipeline = Pipeline([
    ("filter", FilterRaw()),
    ("remove_artifacts", RemoveArtifacts(n_components=8)),
    ("epochify", Epochify(event_ids=event_ids, channels=channels)),
    ("resample", Resampler(sfreq=160)),
])

weibo_subjects = {}

for subject in dataset.subject_list:
    raw = load_single_moabb_subject(dataset, subject)
    raw.drop_channels([ch for ch in raw.ch_names if ch not in channels])

    subject_epochs = weibo_preprocessing_pipeline.fit_transform(raw)

    weibo_subjects[f"W{subject:03}"] = subject_epochs

# Save the preprocessed data in /data/schirrmeister.pkl
import pickle

with open("pickled_data/weibo.pkl", "wb") as f:
    pickle.dump(weibo_subjects, f)

Trial data de-meaned and concatenated with a buffer to create cont data


Applying bandpass filter to raw data...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 331 samples (1.655 s)

Removing artifacts from raw data...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Computing Extended Infomax ICA
Fitting ICA took 74.9s.


f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:69: RuntimeWarning: Using n_components=8 (resulting in n_components_=8) may lead to an unstable mixing matrix estimation because the ratio between the largest (3.4) and smallest (1.6e-29) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 7
  ica.fit(raw)
f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:72: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ica_labels = label_components(raw, ica, method='iclabel')


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'hands', 'left_hand', 'left_hand_right_foot', 'rest', 'right_hand', 'right_hand_left_foot']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
560 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 560 events and 1001 original time points ...
1 bad epochs dropped


Trial data de-meaned and concatenated with a buffer to create cont data


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 1 ICA component
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'hands', 'left_hand', 'left_hand_right_foot', 'rest', 'right_hand', 'right_hand_left_foot']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
560 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 560 events and 1001 original time points ...
1 bad epochs dropped


Trial data de-meaned and concatenated with a buffer to create cont data


Applying bandpass filter to raw data...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 331 samples (1.655 s)

Removing artifacts from raw data...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Computing Extended Infomax ICA
Fitting ICA took 76.5s.


f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:69: RuntimeWarning: Using n_components=8 (resulting in n_components_=8) may lead to an unstable mixing matrix estimation because the ratio between the largest (3.7) and smallest (8.8e-30) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 7
  ica.fit(raw)
f:\zeyad\Documents\repos\general-bci-tl-classifier\scripts\raw_preprocessing.py:72: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ica_labels = label_components(raw, ica, method='iclabel')


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 1 ICA component
    Projecting back using 8 PCA components
Segmenting raw data into epochs...
Used Annotations descriptions: ['feet', 'hands', 'left_hand', 'left_hand_right_foot', 'rest', 'right_hand', 'right_hand_left_foot']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
560 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 560 events and 1001 original time points ...
1 bad epochs dropped
